In [ ]:
from tools import *
from datetime import datetime, timedelta

## Laden der BAG-Daten

In [ ]:
%%time
df_bag_today = load_data()
df_bag_yesterday = load_data(2)

/Users/eandreas/projects/dev/covid-19/bag_data_download/downloads/cases_data/2020-10-23_13-00_Dashboards_1&2_COVID19_swiss_data_pv.xlsx
/Users/eandreas/projects/dev/covid-19/bag_data_download/downloads/cases_data/2020-10-22_13-00_Dashboards_1&2_COVID19_swiss_data_pv.xlsx
CPU times: user 1min 33s, sys: 558 ms, total: 1min 33s
Wall time: 1min 37s


## Neue und bisherige Fälle / Tag

In [ ]:
df = df_bag_today.copy()
index = pd.MultiIndex.from_frame(df[['case_date', 'canton', 'age_class', 'sex']])
df.set_index(index, inplace=True)
df = df.drop(['date', 'time', 'case_date', 'country', 'canton', 'age_class', 'sex', 'date_deceased', 'new_deceased'], axis=1).reset_index()
df = df.groupby(['case_date', 'canton', 'age_class', 'sex']).sum().reset_index()


df_ytd = df_bag_yesterday.copy()
index = pd.MultiIndex.from_frame(df_ytd[['case_date', 'canton', 'age_class', 'sex']])
df_ytd.set_index(index, inplace=True)
df_ytd = df_ytd.drop(['date', 'time', 'case_date', 'country', 'canton', 'age_class', 'sex', 'date_deceased', 'new_deceased'], axis=1).reset_index()
df_ytd = df_ytd.groupby(['case_date', 'canton', 'age_class', 'sex']).sum().reset_index()
df_ytd.rename(columns = {'new_conf': 'new_conf_ytd'}, inplace=True)

df = df.merge(df_ytd, how='left', on=['case_date', 'canton', 'age_class', 'sex'])

### Neue Fälle innerhalb der letzten 24 h - ganze Schweiz

In [ ]:
df.new_conf.sum() - df_ytd.new_conf_ytd.sum()

6634

### Neue Fälle innerhalb der letzten 24 h pro Kanton

In [ ]:
for c in CANTONS.values():
    new_conf_24 = int(df.new_conf[df.canton == c].sum() - df.new_conf_ytd[df.canton == c].sum())
    print(f'{c}: {new_conf_24}')

AG: 291
AI: 19
AR: 38
BE: 553
BL: 90
BS: 81
FL: 42
FR: 469
GE: 950
GL: 15
GR: 116
JU: 71
LU: 186
NE: 204
NW: 20
OW: 7
SG: 382
SH: 24
SO: 97
SZ: 148
TG: 147
TI: 266
UR: 18
VD: 873
VS: 618
ZG: 64
ZH: 845


### Neue Fälle im 7d-Schnitt - ganze Schweiz

In [ ]:
# how many days before totay used as end_date
end_date_offset = 3
# a number >= 1, 1 means consider just one day i.e. the end_date
day_window = 7

end_date = datetime.today() - timedelta(days = end_date_offset)
start_date = end_date - timedelta(days = (day_window - 1))
start_date = start_date.strftime('%Y-%m-%d')
end_date = end_date.strftime('%Y-%m-%d')
print(f'start_date = {start_date}')
print(f'end_date = {end_date}')

df.new_conf[(df.case_date >= start_date) & (df.case_date <= end_date)].sum() / day_window

start_date = 2020-10-18
end_date = 2020-10-24


2998.285714285714

### Neue Fälle im 7d-Schnitt / Kanton

In [ ]:
for c in CANTONS.values():
    new_conf_7d_avg = np.round(df.new_conf[(df.canton == c) & (df.case_date >= start_date) & (df.case_date <= end_date)].sum() / day_window, 1)
    print(f'{c}: {new_conf_7d_avg}')

AG: 130.1
AI: 9.6
AR: 18.9
BE: 271.7
BL: 41.6
BS: 36.4
FL: 14.9
FR: 193.6
GE: 381.3
GL: 7.0
GR: 44.6
JU: 37.9
LU: 104.1
NE: 93.4
NW: 8.9
OW: 4.3
SG: 174.4
SH: 11.7
SO: 45.6
SZ: 69.0
TG: 63.7
TI: 111.9
UR: 7.7
VD: 404.3
VS: 294.0
ZG: 35.4
ZH: 382.4


In [ ]:
### Neue Fälle im 7d-Schnitt pro 100'000 Einwohner und Kanton

In [ ]:
df_pop = pd.read_excel('Population_Size_BFS.xlsx')
df_pop = df_pop.rename(columns = {'ktn': 'canton'})

In [ ]:
for c in CANTONS.values():
    new_conf_7d_avg = df.new_conf[(df.canton == c) & (df.case_date >= start_date) & (df.case_date <= end_date)].sum() / day_window
    new_conf_7d_avg = np.round(new_conf_7d_avg * 100000 / df_pop[df_pop.canton == c].pop_size.sum(), 1)
    print(f'{c}: {new_conf_7d_avg}')

AG: 19.2
AI: 59.3
AR: 34.1
BE: 26.3
BL: 14.4
BS: 18.7
FL: 38.7
FR: 60.7
GE: 76.3
GL: 17.3
GR: 22.5
JU: 51.6
LU: 25.4
NE: 52.8
NW: 20.5
OW: 11.3
SG: 34.4
SH: 14.3
SO: 16.7
SZ: 43.4
TG: 23.0
TI: 31.7
UR: 21.2
VD: 50.6
VS: 85.5
ZG: 27.9
ZH: 25.1


In [ ]:
df_pop[df_pop.canton == 'AG'].pop_size.sum()

678207